# Modelling V2

### This version of the Modelling pipeline is quite different from the previous iteration of the processing system.

In [1]:
import cv2
import pandas as pd

final_data_path = open("step1.txt", "r")
final_data_pre = pd.read_csv(final_data_path.read().strip())

In [2]:
final_data_pre

,id,class_name,file_path,resolution
0,452f2473f3dd5817395dcaa16b58ab6e.png,banana_day1_self,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,720x1280
1,2da7e04b024ef07fd40e4f84213d6ca7.png,banana_day1_self,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,720x1280
2,708f00dd2a7c25d75680d18299d0254d.png,banana_day1_self,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,720x1280
3,866d7784b51be1f26a96af4c99e73448.png,banana_day1_self,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,720x1280
4,2e4fbf3ef0199cd4f664155171dbb849.png,banana_day1_self,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,1280x720
...,...,...,...,...
3821,2790a53203bb55d957f3d51eb8161592.png,avocado_firm_google,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,730x548
3822,d8891a6e393c829c349241f4baac8ad5.png,avocado_firm_google,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,800x378
3823,244c532e45a81497e3d51e81a1fcb04d.png,avocado_firm_google,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,872x534
3824,8109576227fc9d3f8e5b74db76b4da70.png,avocado_firm_google,/home/fadhlan/Normal2/DeepLearningRepo/steps/v...,480x360


In [3]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
import pandas as pd
import numpy as np
import optuna



def load_image(filepath, label):
    img = tf.io.read_file(filepath)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    return img, label

def create_train_test_val_dfs(final_data_pre):
    
    label_processor = tf.keras.layers.StringLookup(
        output_mode='int', 
        vocabulary=final_data_pre['class_name'].unique(),
        num_oov_indices=0,
        mask_token=None 
    )
    number_of_classes = len(final_data_pre['class_name'].unique())
    train_and_tuning_df, val_df = train_test_split(
        final_data_pre, 
        test_size=0.15, 
        random_state=42, 
        stratify=final_data_pre['class_name']
    )
    train_df, test_df = train_test_split(
        train_and_tuning_df, 
        test_size=0.2, 
        random_state=42, 
        stratify=train_and_tuning_df['class_name']
    )
    print(f"Train (Weights):    {len(train_df)}")
    print(f"Test  (Optuna):     {len(test_df)}")
    print(f"Val   (Final Eval): {len(val_df)}")
    return {
        "number_of_classes": number_of_classes,
        "label_processor": label_processor,
        "train_df": train_df,
        "test_df": test_df,
        "val_df": val_df
    }

2025-12-16 15:23:13.555115: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/root/miniconda3/envs/DeepLearning/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from tensorflow.keras import layers, Model, regularizers

def create_dses(input):
    # --- Configuration ---
    data_augmentation = tf.keras.Sequential([
        layers.RandomFlip("horizontal_and_vertical"),
        layers.RandomRotation(0.2), # Rotate +/- 20%
        layers.RandomZoom(0.2),     # Zoom in/out 20%
        layers.RandomTranslation(0.1, 0.1), # Shift image
        layers.RandomContrast(0.2), # Adjust contrast
        layers.RandomBrightness(0.2)
    ])

    def load_image(filepath, label):
        img = tf.io.read_file(filepath)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [224, 224])
        return img, label

    def create_dataset(df, is_training=True, batch_size=32):
        # Create source dataset
        ds = tf.data.Dataset.from_tensor_slices((df['file_path'].values, df['class_name'].values))
        ds = ds.map(lambda x, y: (x, input["label_processor"](y)))
        ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
        if is_training:
            ds = ds.shuffle(buffer_size=1000)
            # We wrap augmentation in a lambda to ensure it runs correctly in graph mode
            ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                        num_parallel_calls=tf.data.AUTOTUNE)
        ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return ds

    # Create the specific datasets
    train_ds = create_dataset(input["train_df"], is_training=True)
    test_ds  = create_dataset(input["test_df"], is_training=False)
    val_ds   = create_dataset(input["val_df"], is_training=False)
    return {
        "train_ds": train_ds,
        "number_of_classes": input["number_of_classes"],
        "test_ds": test_ds,
        "val_ds": val_ds
    }

In [5]:
%run modeldef3.py

def train_model(train_ds, test_ds, number_of_classes, model_name, number_of_trials):
    def objective(trial):
        # 1. Suggest Hyperparameters
        # Log scale is good for learning rates
        lr = trial.suggest_float('learning_rate', 1e-5, 1e-3, log=True) 
        
        # Suggest units for layers
        u1 = trial.suggest_int('units_1', 64, 512, step=32)
        u2 = trial.suggest_int('units_2', 32, 256, step=32)
        u3 = trial.suggest_int('units_3', 32, 128, step=16)
        
        # Suggest dropout
        dr = trial.suggest_float('dropout', 0.1, 0.5)
        epochs = trial.suggest_int("epochs", 5,20, step=1)
        
        # 2. Build Model
        model = TunableResNetClassifier(
            num_classes=number_of_classes,
            units_1=u1,
            units_2=u2,
            units_3=u3,
            dropout_rate=dr
        )
        
        # 3. Compile
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )
        
        # 4. Train with Pruning
        # The PruningCallback will stop unpromising trials early to save time
        history = model.fit(
            train_ds,
            validation_data=test_ds,
            epochs=epochs, # Short epochs for tuning (increase for final training)
            verbose=0, # Keep output clean
            callbacks=[optuna.integration.TFKerasPruningCallback(trial, 'val_accuracy')]
        )
        
        # Return the best validation accuracy of this trial
        return max(history.history['val_accuracy'])
    storage_name = "sqlite:///{}.db".format(model_name)

    study = optuna.create_study(
        study_name=model_name,
        storage=storage_name,  
        direction='maximize',
        load_if_exists=True    
    )

    print("Starting Hyperparameter Tuning for " + model_name + "....")
    study.optimize(objective, n_trials=number_of_trials)

    print("Best trial:")
    trial = study.best_trial

    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
    return study

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


def train_final_and_evaluate(study, train_ds, test_ds, val_ds, number_of_classes):
    best_params = study.best_trial.params
    print("Best Parameters Found:")
    print(best_params)

    # train the final BEST model with the BEST parameters
    final_model = TunableResNetClassifier(
        num_classes=number_of_classes,
        units_1=best_params['units_1'],
        units_2=best_params['units_2'],
        units_3=best_params['units_3'],
        dropout_rate=best_params['dropout']
    )

    # Compile model with final learning rate
    final_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    # Final Training Run
    print("\nStarting Final Training...")
    history = final_model.fit(
        train_ds,
        validation_data=test_ds, 
        epochs=best_params['epochs'] + 5, # minor fudge factor for epochs
        verbose=1
    )

    # The Moment of Truth (Evaluation)
    # This is the ONLY time 'val_ds' is touched.
    print("\nRunning Final Evaluation on Hold-out Set (val_ds)...")
    test_loss, test_accuracy = final_model.evaluate(val_ds)

    print("="*40)
    print(f"FINAL UNBIASED ACCURACY: {test_accuracy * 100:.2f}%")
    print("="*40)
    return {
        "final_model": final_model,
        "history": history
    }

def visualize_predictions(dataset, model, label_processor, grid_rows=2, grid_cols=4):
    
    # Calculate total samples needed per category
    num_samples = grid_rows * grid_cols
    class_names = label_processor.get_vocabulary()
    
    # Containers
    correct_batch = {'imgs': [], 'true': [], 'pred': []}
    incorrect_batch = {'imgs': [], 'true': [], 'pred': []}
    
    # 1. Collection Loop
    print(f"Collecting {num_samples} correct and {num_samples} incorrect examples...")
    
    for images, labels in dataset:
        # Stop if we have enough of both
        if (len(correct_batch['imgs']) >= num_samples and 
            len(incorrect_batch['imgs']) >= num_samples):
            break

        logits = model(images, training=False)
        preds = tf.argmax(logits, axis=1)
        
        images_np = images.numpy()
        labels_np = labels.numpy()
        preds_np = preds.numpy()
        
        for i in range(len(images_np)):
            if (len(correct_batch['imgs']) >= num_samples and 
                len(incorrect_batch['imgs']) >= num_samples):
                break
            
            true_idx = labels_np[i]
            pred_idx = preds_np[i]
            is_correct = (true_idx == pred_idx)
            
            target_dict = correct_batch if is_correct else incorrect_batch
            
            if len(target_dict['imgs']) < num_samples:
                target_dict['imgs'].append(images_np[i])
                target_dict['true'].append(class_names[true_idx])
                target_dict['pred'].append(class_names[pred_idx])

    # 2. Grid Plotting Logic
    def plot_grid(data_dict, title, color_code):
        count = len(data_dict['imgs'])
        if count == 0:
            print(f"No {title.lower()} predictions found.")
            return

        # Create the figure with subplots
        # figsize is adjusted dynamically: width = 3*cols, height = 3*rows
        fig, axes = plt.subplots(grid_rows, grid_cols, figsize=(3 * grid_cols, 3.5 * grid_rows))
        fig.suptitle(f"{title} Predictions", fontsize=20, color=color_code, weight='bold', y=1.02)
        
        # Flatten axes array for easy iteration (handles 1D and 2D arrays automatically)
        axes_flat = axes.flatten() if count > 1 else [axes]
        
        for i in range(num_samples):
            ax = axes_flat[i]
            
            # If we ran out of data (e.g., model is 100% accurate, so 0 incorrect), hide axis
            if i >= count:
                ax.axis('off')
                continue
            
            # Plot Image
            img = data_dict['imgs'][i].astype("uint8")
            ax.imshow(img)
            ax.axis("off")
            
            # Plot Label
            true_lab = data_dict['true'][i]
            pred_lab = data_dict['pred'][i]
            
            # Bold the label if it matches the title type
            label_text = f"True: {true_lab}\nPred: {pred_lab}"
            ax.set_title(label_text, color='black', fontsize=11, pad=10)
            
            # Add a colored border to the subplot to indicate status
            # border_color = color_code
            # plt.setp(ax.spines.values(), color=border_color, linewidth=2)

        plt.tight_layout()
        plt.show()

    # 3. Generate the two grids
    plot_grid(correct_batch, "Correct", "green")
    print("\n" + "="*50 + "\n")
    plot_grid(incorrect_batch, "Incorrect", "red")
    # Get class names from the processor to decode predictions later
    class_names = label_processor.get_vocabulary()
    
    # Containers
    correct_batch = {'imgs': [], 'true': [], 'pred': []}
    incorrect_batch = {'imgs': [], 'true': [], 'pred': []}
    
    # 1. Iterate through the dataset
    for images, labels in dataset:
        # Get model predictions (Logits)
        logits = model(images, training=False)
        preds = tf.argmax(logits, axis=1) # Convert logits to class ID
        
        # Convert tensors to numpy for easy handling
        images_np = images.numpy()
        labels_np = labels.numpy()
        preds_np = preds.numpy()
        
        # 2. Sort images into Correct vs Incorrect
        for i in range(len(images_np)):
            # Check if we have collected enough of both types
            if (len(correct_batch['imgs']) >= num_samples and 
                len(incorrect_batch['imgs']) >= num_samples):
                break
            
            true_idx = labels_np[i]
            pred_idx = preds_np[i]
            
            # Helper to store data
            is_correct = (true_idx == pred_idx)
            target_dict = correct_batch if is_correct else incorrect_batch
            
            # Only append if we haven't reached the limit for this type
            if len(target_dict['imgs']) < num_samples:
                target_dict['imgs'].append(images_np[i])
                target_dict['true'].append(class_names[true_idx])
                target_dict['pred'].append(class_names[pred_idx])
        
        # Break outer loop if full
        if (len(correct_batch['imgs']) >= num_samples and 
            len(incorrect_batch['imgs']) >= num_samples):
            break

    # 3. Plotting Logic
    def plot_row(data_dict, title, color):
        count = len(data_dict['imgs'])
        if count == 0:
            print(f"No {title.lower()} predictions found to display.")
            return

        plt.figure(figsize=(15, 4))
        plt.suptitle(f"{title} Predictions", fontsize=16, color=color, weight='bold')
        
        for i in range(count):
            plt.subplot(1, count, i + 1)
            
            # Image is likely float32 (0-255), cast to uint8 for imshow
            img = data_dict['imgs'][i].astype("uint8")
            plt.imshow(img)
            plt.axis("off")
            
            # Title: True vs Pred
            true_lab = data_dict['true'][i]
            pred_lab = data_dict['pred'][i]
            
            label_text = f"True: {true_lab}\nPred: {pred_lab}"
            plt.title(label_text, color=color, fontsize=10)
        plt.show()

    # Show Correct Row (Green)
    plot_row(correct_batch, "Correct", "green")
    
    # Show Incorrect Row (Red)
    plot_row(incorrect_batch, "Incorrect", "red")

## Avocado Testing & Evaluation

In [ ]:

# Train avocado only model
avocado_only = final_data_pre[final_data_pre["class_name"].str.contains("avocado")]
step1avc = create_train_test_val_dfs(avocado_only)
step2avc = create_dses(step1avc)




Train (Weights):    2368
Test  (Optuna):     592
Val   (Final Eval): 523


[I 2025-12-16 14:56:13,453] Using an existing study with name 'avocado_only' instead of creating a new one.


In [ ]:
study_avc = train_model(step2avc["train_ds"],step2avc["test_ds"], step2avc["number_of_classes"], "avocado_only", 10)

In [19]:
storage_name = "sqlite:///{}.db".format("avocado_only")

study = optuna.create_study(
    study_name="avocado_only",
    storage=storage_name,  
    direction='maximize',
    load_if_exists=True    
)
study_avc = study

[I 2025-12-16 15:02:13,253] Using an existing study with name 'avocado_only' instead of creating a new one.


In [ ]:
avc_results = train_final_and_evaluate(study_avc, step2avc["train_ds"],step2avc["test_ds"], step2avc["val_ds"], step2avc["number_of_classes"])


In [ ]:
visualize_predictions(
    step2avc["val_ds"],
    avc_results["final_model"],
    step1avc["label_processor"],
    grid_rows=3,  # Adjust rows here
    grid_cols=4   # Adjust columns here
)

## Banana Testing & Evaluation

In [23]:

# Train banana only model
banana_only = final_data_pre[final_data_pre["class_name"].str.contains("banana")]
step1banan = create_train_test_val_dfs(banana_only)
step2banan = create_dses(step1banan)

Train (Weights):    232
Test  (Optuna):     59
Val   (Final Eval): 52


In [ ]:
study_banana = train_model(step2banan["train_ds"],step2banan["test_ds"], step2banan["number_of_classes"], "banana_only", 10)

[I 2025-12-16 15:18:15,098] A new study created in RDB with name: banana_only


Starting Hyperparameter Tuning for banana_only....


2025-12-16 15:18:23.295508: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-16 15:18:23.295536: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-16 15:18:23.295546: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-16 15:18:23.295563: I external/l

Best trial:
  Value: 0.5593220591545105
  Params: 
    learning_rate: 0.00033980689906860624
    units_1: 192
    units_2: 32
    units_3: 80
    dropout: 0.3640985193285784
    epochs: 8


In [ ]:
storage_name = "sqlite:///{}.db".format("banana_only")

study = optuna.create_study(
    study_name="banana_only",
    storage=storage_name,  
    direction='maximize',
    load_if_exists=True    
)
study_banana = study

In [25]:
banana_results = train_final_and_evaluate(study_banana, step2banan["train_ds"],step2banan["test_ds"], step2banan["val_ds"], step2banan["number_of_classes"])

Best Parameters Found:
{'learning_rate': 0.00033980689906860624, 'units_1': 192, 'units_2': 32, 'units_3': 80, 'dropout': 0.3640985193285784, 'epochs': 8}

Starting Final Training...
Epoch 1/13
8/8 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.1207 - loss: 2.1831 - val_accuracy: 0.2203 - val_loss: 1.9244
Epoch 2/13
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - accuracy: 0.1595 - loss: 1.9621 - val_accuracy: 0.2881 - val_loss: 1.9088
Epoch 3/13
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.1897 - loss: 1.9364 - val_accuracy: 0.2712 - val_loss: 1.8959
Epoch 4/13
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - accuracy: 0.1983 - loss: 1.9203 - val_accuracy: 0.3220 - val_loss: 1.8221
Epoch 5/13
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.2155 - loss: 1.8755 - val_accuracy: 0.2712 - val_loss: 1.7395
Epoch 6/13
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - accuracy: 0.2629 - loss: 1.8100 - val_accuracy: 0.3220 - val_loss: 1.6567
Epoch 7/13
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - accuracy: 0.

2025-12-16 15:22:03.437564: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1730', 52 bytes spill stores, 52 bytes spill loads



2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5000 - loss: 1.1795  
FINAL UNBIASED ACCURACY: 50.00%


In [26]:
visualize_predictions(
    step2banan["val_ds"],
    banana_results["final_model"],
    step1banan["label_processor"],
    grid_rows=3,  # Adjust rows here
    grid_cols=4   # Adjust columns here
)

2025-12-16 15:22:24.367781: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 49.00MiB (rounded to 51380224)requested by op Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-12-16 15:22:24.368460: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1049] BFCAllocator dump for GPU_0_bfc
2025-12-16 15:22:24.368485: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1056] Bin (256): 	Total Chunks: 3674, Chunks in use: 3673. 918.5KiB allocated for chunks. 918.2KiB in use in bin. 266.0KiB client-requested in use in bin.
2025-12-16 15:22:24.368490: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1056] Bin (512): 	Total Chunks: 1237, Chunks in use: 1236. 631.2KiB allocated for chunks. 630.8KiB in use in bin. 618.4KiB client-requested in use i

ResourceExhaustedError: Exception encountered when calling Conv2D.call().

[1m{{function_node __wrapped__Conv2D_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[32,28,28,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D][0m

Arguments received by Conv2D.call():
  • inputs=tf.Tensor(shape=(32, 28, 28, 128), dtype=float32)

## Combined Model Testing & Evaluation

In [7]:

# Train banana only model
all_in_one = final_data_pre
step1aio = create_train_test_val_dfs(all_in_one)
step2aio = create_dses(step1aio)

I0000 00:00:1765898606.572992 2315936 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6136 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:01:00.0, compute capability: 8.9


Train (Weights):    2601
Test  (Optuna):     651
Val   (Final Eval): 574


In [ ]:
study_aio = train_model(step2aio["train_ds"],step2aio["test_ds"], step2aio["number_of_classes"], "all_in_one", 10)

[I 2025-12-16 15:23:28,545] Using an existing study with name 'all_in_one' instead of creating a new one.


Starting Hyperparameter Tuning for all_in_one....


2025-12-16 15:23:36.625246: I external/local_xla/xla/service/service.cc:163] XLA service 0x7fa21c05a960 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-12-16 15:23:36.625271: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4060, Compute Capability 8.9
2025-12-16 15:23:36.964207: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-16 15:23:38.334465: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002
2025-12-16 15:23:38.601601: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-12-16 15:23:38.601630: I e

Best trial:
  Value: 0.9093701839447021
  Params: 
    learning_rate: 6.018284784261046e-05
    units_1: 320
    units_2: 128
    units_3: 128
    dropout: 0.2184349600734361
    epochs: 16


In [ ]:
storage_name = "sqlite:///{}.db".format("all_in_one")

study = optuna.create_study(
    study_name="all_in_one",
    storage=storage_name,  
    direction='maximize',
    load_if_exists=True    
)
study_aio = study

In [ ]:
aio_results = train_final_and_evaluate(study_aio, step2aio["train_ds"],step2aio["test_ds"], step2aio["val_ds"], step2aio["number_of_classes"])

In [ ]:
visualize_predictions(
    step2aio["val_ds"],
    aio_results["final_model"],
    step1aio["label_processor"],
    grid_rows=3,  # Adjust rows here
    grid_cols=4   # Adjust columns here
)